In [3]:
import pandas as pd
melb_data = pd.read_csv('data/melb_data.csv', sep=',')

Под фильтрацией в DataFrame подразумевается получение новой таблицы путём вырезания строк, не удовлетворяющих поставленному условию

### фильтрацию с помощью масок

Маской называется Series, которая состоит из булевых значений, при этом значения True соответствуют тем индексам, для которых заданное условие выполняется, в противном случае ставится значение False (например, цена > 2 млн).

In [8]:
# Создадим маску и положим её в переменную с именем mask. Синтаксис очень прост:

mask = melb_data['Price'] > 2000000

display(mask)

0        False
1        False
2        False
3        False
4        False
         ...  
13575    False
13576    False
13577    False
13578     True
13579    False
Name: Price, Length: 13580, dtype: bool

In [9]:
# Для фильтрации нужно просто подставить переменную mask в индексацию DataFrame. 
# Маска показывает, какие строки нужно оставлять в результирующем наборе, а какие — убирать 
# (выведем первые пять строк отфильтрованной таблицы):

melb_data[mask].head(5)

,index,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Coordinates
80,80,Albert Park,112 Beaconsfield Pde,3,h,2850000.0,PI,Buxton,4/03/2017,3.3,...,0.0,211.0,198.0,1890.0,Port Phillip,-37.8481,144.9499,Southern Metropolitan,3280.0,"-37.8481, 144.9499"
85,85,Albert Park,104 Richardson St,4,h,2300000.0,S,Marshall,7/05/2016,3.3,...,1.0,153.0,180.0,1880.0,Port Phillip,-37.8447,144.9523,Southern Metropolitan,3280.0,"-37.8447, 144.9523"
88,88,Albert Park,29 Faussett St,2,h,2120000.0,S,Greg,10/09/2016,3.3,...,1.0,199.0,107.0,1900.0,Port Phillip,-37.8422,144.9554,Southern Metropolitan,3280.0,"-37.8422, 144.9554"
92,92,Albert Park,2 Dundas Pl,3,h,2615000.0,S,Cayzer,10/12/2016,3.3,...,1.0,177.0,181.0,1880.0,Port Phillip,-37.8415,144.9585,Southern Metropolitan,3280.0,"-37.8415, 144.9585"
93,93,Albert Park,23 Finlay St,5,h,2100000.0,S,Greg,10/12/2016,3.3,...,1.0,237.0,126.0,1970.0,Port Phillip,-37.8436,144.9557,Southern Metropolitan,3280.0,"-37.8436, 144.9557"


**Примечание**. В результате выполнения фильтрации возвращается новый DataFrame, полученный из исходного, при этом исходная таблица melb_data остаётся без изменений.

Также вовсе не обязательно заносить маску в отдельную переменную — можно сразу вставлять условие в операцию индексации DataFrame, например:

melb_data[melb_data['Price'] > 2000000]


In [15]:
#Найдём количество зданий с тремя комнатами. 
# Для этого отфильтруем таблицу по условию: обратимся к результирующей таблице по столбцу Rooms и 
# найдём число строк в ней с помощью атрибута shape:

melb_data[melb_data['Rooms'] == 3].shape[0]

5881

In [16]:
#Усложним прошлый пример и найдём число трёхкомнатных домов с ценой менее 300 тысяч:

melb_data[(melb_data['Rooms'] == 3) & (melb_data['Price'] < 300000)].shape[0]

3

In [17]:
#Таких зданий оказалось всего три. 
# Немного «ослабим» условие: теперь нас будут интересовать дома с ценой менее 300 тысяч, у которых 
# либо число комнат равно 3 
# либо площадь домов более 100 квадратных метров:

melb_data[((melb_data['Rooms'] == 3) | (melb_data['BuildingArea'] > 100)) & (melb_data['Price'] < 300000)].shape[0]

68

**Примечание**. Обратите внимание, что использование привычных операторов **and** и **or** будет неверным и приведёт к ошибке, так как они выполняют логические операции между двумя булевыми числами. В нашем случае слева и справа от оператора стоят маски (объекты Series), для которых логическую операцию надо совершить поэлементно, а операторы **and** и **or** для такого не предназначены.

### Фильтрацию часто сочетают со статистическими методами.

In [18]:
#Давайте найдём максимальное количество комнат в таунхаусах. 
# Так как в результате фильтрации получается DataFrame, 
# то обратимся к нему по столбцу Rooms и найдём максимальное значение:

melb_data[melb_data['Type'] == 't']['Rooms'].max()

5

In [19]:
#А теперь более сложный трюк: найдём медианную площадь здания у объектов, чья цена выше средней. 
# Для того чтобы оградить наш код от нагромождений, предварительно создадим переменную со средней ценой:

mean_price = melb_data['Price'].mean()
melb_data[melb_data['Price'] > mean_price]['BuildingArea'].median()

126.0

Фильтрация находит применение в очистке данных.

Под очисткой данных понимается удаление из данных аномальных значений (выбросов), пропусков и данных, которые не несут информацию.

Причин появления таких дефектов в данных множество: неправильная передача по сети, неверная выгрузка из базы, человеческий фактор и т. д.

→ Проблема аномальных значений состоит в том, что если мы обучаем модель на данных, содержащих выбросы, то мы имеем немалый шанс получить менее верный прогноз, чего нам, конечно же, не хотелось бы.

In [38]:
#У скольких объектов недвижимости из таблицы melb_data отсутствуют ванные комнаты?
print(melb_data[melb_data['Bathroom']==0].shape[0])

#Сколько в таблице melb_data объектов недвижимости, которые были проданы риелторской компанией Nelson 
# и стоимость которых составила больше 3 миллионов?

melb_data[(melb_data['SellerG'] == 'Nelson') & (melb_data['Price'] > 3000000)].shape[0]


#Какова минимальная стоимость участка без здания (площадь здания равна 0) в таблице melb_data?

melb_data[melb_data['BuildingArea'] == 0]['Price'].min()

#Какова средняя цена объектов недвижимости в таблице melb_data 
# с ценой менее одного миллиона, в которых 
# либо количество комнат больше пяти, 
# либо здание моложе 2015 года?

round(melb_data[(melb_data['Price'] < 1000000) & ((melb_data['Rooms'] > 5)|(melb_data['YearBuilt'] > 2015))]['Price'].mean())



#В каком районе Мельбурна чаще всего продаются виллы и коттеджи (тип здания — h) с ценой меньше трёх миллионов?

melb_data[(melb_data['Type'] == 'h') & (melb_data['Price'] < 3000000)]['Regionname'].mode()


34


0    Northern Metropolitan
Name: Regionname, dtype: object